In [1]:
import ipaddress
from datetime import date, datetime

import pandas as pd
from metar import Metar
from metarparse.files import get_metar_observations

from aimlsse_api.client import GroundDataClient

In [2]:
ground_data_client = GroundDataClient(ipaddress.ip_address('127.0.0.1'), 8000)

In [3]:
data = ground_data_client.queryMetar(['EDDE','EDDF'], date(2022, 10, 1), date(2022, 12, 1))
print(data)

     station            datetime  \
0       EDDE 2022-10-01 00:20:00   
1       EDDE 2022-10-01 00:50:00   
2       EDDE 2022-10-01 01:20:00   
3       EDDE 2022-10-01 01:50:00   
4       EDDE 2022-10-01 02:20:00   
...      ...                 ...   
5850    EDDF 2022-11-30 21:50:00   
5851    EDDF 2022-11-30 22:20:00   
5852    EDDF 2022-11-30 22:50:00   
5853    EDDF 2022-11-30 23:20:00   
5854    EDDF 2022-11-30 23:50:00   

                                                  metar  
0           EDDE 010020Z AUTO 19008KT CAVOK 08/05 Q1011  
1           EDDE 010050Z AUTO 19009KT CAVOK 08/05 Q1010  
2           EDDE 010120Z AUTO 19009KT CAVOK 08/05 Q1010  
3           EDDE 010150Z AUTO 19009KT CAVOK 08/05 Q1010  
4           EDDE 010220Z AUTO 19011KT CAVOK 09/06 Q1010  
...                                                 ...  
5850  COR EDDF 302150Z AUTO 11005KT 9999 -DZ OVC013 ...  
5851  COR EDDF 302220Z AUTO 15004KT 9999 OVC012 06/0...  
5852  EDDF 302250Z AUTO 13003KT 9999 -DZ BKN0

In [4]:
def decode_metar(metar_data:str, obs_datetime:datetime) -> Metar.Metar:
    result = Metar.Metar(metar_data, month=obs_datetime.month, year=obs_datetime.year, strict=False)
    return result

In [5]:
data['decoded_metar'] = data.apply(lambda row: decode_metar(row.metar, row.datetime), axis=1)

c:\Users\erikw\anaconda3\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' while processing 'EDDE 080850Z AUTO 17010KT CAVOK 12/07 Q1013 RE// NOSIG'
  warnings.warn(message, RuntimeWarning)
c:\Users\erikw\anaconda3\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'BKN00A' while processing 'EDDE 211620Z AUTO 01003KT 340V050 3400 -FZDZ BR FEW020 BKN031 M00/M00 Q1001 TEMPO 0200 FZFG BKN00A'
  warnings.warn(message, RuntimeWarning)
c:\Users\erikw\anaconda3\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' while processing 'EDDF 110850Z AUTO 03009KT 350V080 CAVOK 12/07 Q1027 RE// NOSIG'
  warnings.warn(message, RuntimeWarning)
c:\Users\erikw\anaconda3\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'RE//' while processing 'EDDF 180850Z AUTO 27004KT 240V300 9999 FEW005 17/16 Q1026 RE// TEMPO BKN005'
  warnings.warn(message, RuntimeWarning)
c:\Users\erikw\anaconda3\

In [9]:
for metar in data['decoded_metar']:
    assert isinstance(metar, Metar.Metar)
    if metar.weather:
        print(f'station: {metar.station_id}, weather: {metar.weather}')

station: EDDE, weather: [('-', None, 'RA', None, None)]
station: EDDE, weather: [('-', None, 'RA', None, None)]
station: EDDE, weather: [('-', None, 'RA', None, None)]
station: EDDE, weather: [('-', None, 'RA', None, None)]
station: EDDE, weather: [('-', None, 'RA', None, None)]
station: EDDE, weather: [('-', None, 'DZ', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('VC', 'SH', '', None, None)]
station: EDDE, weather: [('VC', 'SH', '', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, None)]
station: EDDE, weather: [('-', 'SH', 'RA', None, No

In [6]:
variables = ['station_id', 'time', 'cycle', 'type', 'mod', 
             'visibility_m', 'temperature_C', 'dew_point_C', 'wind_speed_kmh', 'wind_dir', 
             'pressure_mb', 'sea_level_pressure_mb', 'snowdepth_m',
             'lowcloud_cover', 'lowcloud_height_m', 'lowcloud_type',
             'CAVOK', 'code']

In [7]:
entries = (get_metar_observations(x, variables) for x in data['decoded_metar'])
observations = pd.DataFrame(entries, columns=variables)
print(observations)

     station_id                time  cycle   type   mod  visibility_m  \
0          EDDE 2022-10-01 00:20:00      0  METAR  AUTO       10000.0   
1          EDDE 2022-10-01 00:50:00      1  METAR  AUTO       10000.0   
2          EDDE 2022-10-01 01:20:00      1  METAR  AUTO       10000.0   
3          EDDE 2022-10-01 01:50:00      2  METAR  AUTO       10000.0   
4          EDDE 2022-10-01 02:20:00      2  METAR  AUTO       10000.0   
...         ...                 ...    ...    ...   ...           ...   
5850       EDDF 2022-11-30 21:50:00     22  METAR  AUTO       10000.0   
5851       EDDF 2022-11-30 22:20:00     22  METAR  AUTO       10000.0   
5852       EDDF 2022-11-30 22:50:00     23  METAR  AUTO       10000.0   
5853       EDDF 2022-11-30 23:20:00     23  METAR  AUTO       10000.0   
5854       EDDF 2022-11-30 23:50:00     24  METAR  AUTO        8000.0   

      temperature_C  dew_point_C  wind_speed_kmh  wind_dir  pressure_mb  \
0               8.0          5.0       14.815987